In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install git+https://github.com/APCLab/jieba-tw.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/APCLab/jieba-tw.git to /tmp/pip-req-build-h2rbl5e4
  Running command git clone -q https://github.com/APCLab/jieba-tw.git /tmp/pip-req-build-h2rbl5e4
  Created wheel for jieba: filename=jieba-0.39.1-py3-none-any.whl size=6603877 sha256=ea93cb469d5a9d93aa113aa917c832e9c549837398a353104c86151c37b29118
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ikztsx7/wheels/f3/92/35/5e2679360de6fd4c424678c139718c898f0a87ae600458aa76
Successfully built jieba
  Attempting uninstall: jieba
    Found existing installation: jieba 0.42.1
    Uninstalling jieba-0.42.1:
      Successfully uninstalled jieba-0.42.1


## 設定目前程式資料夾路徑(務必自行更改)

In [2]:
folder="/content/drive/MyDrive/解密AI黑盒子分享/"

## import 基礎套件

In [3]:
import jieba
import random
import numpy as np
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")

## 載入pytorch相關套件

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from gensim.models import word2vec

## 是否有GPU可以使用

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 讀取資料集檔案

In [17]:
with open("/content/drive/MyDrive/解密AI黑盒子分享/資料集/外賣評價.txt") as f:
    file_list = f.readlines()
data=[line.strip().replace(",","").replace("，","").replace("。","").replace("！","") for line in file_list[1:]]
print(data)

['1很快好吃味道足量大', '1沒有送水沒有送水沒有送水', '1非常快態度好', '1方便快捷味道可口快遞給力', '1菜味道很棒送餐很及時', '1今天師傅是不是手抖了微辣格外辣', '1"送餐快態度也特別好辛苦啦謝謝"', '1超級快就送到了這麼冷的天氣騎士們辛苦了謝謝你們麻辣香鍋依然很好吃', '1經過上次晚了2小時這次超級快20分鐘就送到了……', '1最後五分鐘訂的賣家特別好接單了謝謝', '1量大好吃每次點的都夠吃兩次', '1挺辣的吃著還可以吧', '1味道好送餐快分量足', '1量足好吃送餐也快', '1特別好吃量特大而且送餐特別快特別特別棒', '1口感好的很速度快', '1相當好吃的香鍋分量夠足味道也沒的說', '1好吃速度包裝也有品質不出家門就能吃到餐廳的味道', '1味道好極啦送餐很快師傅辛苦啦', '1量大味道好送餐師傅都很好', '1送餐師傅很好味道好極啦', '1送貨速度很快一直定這家贊', '1很方便很快就送到了棒', '1好吃總點這麼多夠五個人吃送的很快', '1"很香很美味下次還會光顧"', '1"送餐特別快態度也好辛苦啦"', '1服務很不錯送到的很快半小時不到就送來了', '1速度很快大霧霾天外賣騎士態度都很好讚讚贊', '1味道正宗量大內容多', '1"送餐非常快態度特別好謝謝"', '1又快又好量足經常吃', '1好大一盆點了7個小份量足', '1配送人員態度好速度快', '1"在這種天氣裡感謝送餐員的辛苦服務謝謝啦"', '1"送餐特別快態度好非常感謝"', '1送的非常快包裝好謝謝師傅', '1附近最好吃的麻辣香鍋不開玩笑的', '1味道不錯份量很足建議都點小份紅薯超好吃就是太爛了容易碎', '1還不錯就是稍微鹹了點', '1這麼晚辛苦外賣小哥了', '1超級快就送到了謝謝騎士很快感謝騎士這種天氣還在工作', '1非常好吃味道也很香推薦', '1"很好吃速遞快下次繼續選擇"', '1很快特別好', '1太麻了青筍有點小米飯給的也不多土豆片都碎了找不到了', '1點了太多次了味道很香', '1"態度很好地址填錯了還是給我跑了一趟沒有表現出不願意的樣子為了這個快遞員我寫了評論"', '1快遞小哥很快就送到了贊水煮牛肉肉質鮮嫩辣的恰到好處也很入味不錯挺好吃的', '1"口味不錯乾淨味道好送貨員服務非常好"', '1"送餐

In [20]:
with open("/content/drive/MyDrive/解密AI黑盒子分享/資料集/外賣評價.txt") as f:
    file_list = f.readlines()
data=[line.strip().replace(",","").replace("，","").replace("。","").replace("！","") for line in file_list[1:]]
for i in range(20):
  random.shuffle(data)
data_x=[jieba.lcut(text[1:]) for text in data]
data_y=[text[0] for text in data]
print(data_x)
print(data_y)

[['送餐', '速度', '太慢', '了', '真是', '不能', '忍了', '好', '脾氣', '全', '被', '磨', '沒', '了'], ['太慢', '了', '都', '餓死', '了', '大', '熱天', '送過來', '飯菜', '都', '涼', '了', '？', '？'], ['我', '該', '怎麼', '評價', '？', '？', '諾大', '的', '盒子', '…', '…', '只', '裝了', '半盒', '…', '…', '3', '塊', '錢', '一', '份', '的', '鵪鶉蛋', '只有', '2', '顆', '…', '…', '我', '忍了', '…', '…', '好歹', '你', '給', '我', '放了', '問題', '是', '我', '要', '的', '豆皮', '呢', '？', '？', '？', '我', '要', '的', '苦菊', '呢', '？', '？', '？', '裡面', '綠色', '的', '菜葉', '我', '用', '一', '根', '筷子', '就', '能', '挑起來', '…', '…', '3', '塊', '錢', '一', '份', '的', '寬粉', '…', '…', '只有', '中指', '那麼', '長', '？', '？', '太', '垃圾', '誰', '點', '你們', '家', '外賣', '誰', '死', '全家', '的', '百度', '優惠', '完', '還', '30', '塊', '錢', '呢', '這麼', '大', '的', '盒子', '只有', '半盒', '不', '說', '…', '…', '去', '了', '湯', '還', '剩', '什麼', '？', '？', '垃圾', '垃圾', '垃圾', '垃圾', '垃圾', '垃圾', '垃圾', '垃圾', '垃圾', '垃圾', '…', '…', '一', '條', '寬粉', '兩', '顆', '鵪鶉蛋', '…', '…', '黑白', '各', '一', '塊', '百葉', '…', '…', '還', '有', '一', '支', '筷子', '就', '能', '挑起來', '的', '

## 使用Word2Vec做word Embedding 訓練

In [21]:
model_w2v = word2vec.Word2Vec(data_x, size=128,iter=10,sg=1,min_count=0)
model_w2v.save('test_word2vec.model')

In [22]:
print(model_w2v)

Word2Vec(vocab=11506, size=128, alpha=0.025)


In [23]:
print(len(model_w2v.wv.vocab))

11506


In [78]:
model_w2v.wv.get_vector("讚")

array([-6.33139685e-02,  9.61899459e-02, -1.08653285e-01, -1.78035527e-01,
       -6.24795482e-02, -2.10052222e-01, -3.52589220e-01,  1.64494097e-01,
        1.07131526e-02, -2.40692466e-01,  1.78785592e-01, -2.74236295e-02,
        4.23305028e-04,  1.98663458e-01, -1.26727195e-02,  3.26991111e-01,
        4.72421665e-03,  6.28156736e-02, -4.03749466e-01,  3.40612710e-01,
        5.71612902e-02, -5.35898143e-03,  2.93213904e-01,  2.64039755e-01,
       -1.52344495e-01, -1.90632623e-02, -4.72832993e-02,  3.36069427e-02,
        2.57158488e-01, -3.39644730e-01, -2.04786986e-01, -4.90050912e-01,
        9.39067006e-02, -1.15816660e-01,  7.34437332e-02,  1.47669435e-01,
       -4.34047244e-02,  9.10416096e-02,  5.48062958e-02,  2.55547851e-01,
        1.32058606e-01, -2.75137663e-01,  3.44243944e-01,  4.07356977e-01,
       -2.90267587e-01,  1.72331899e-01,  5.74761890e-02,  2.79834598e-01,
       -4.36523825e-01, -1.82223395e-01,  1.34826690e-01, -1.70557544e-01,
       -8.57413337e-02, -

In [79]:
model_w2v.wv.similar_by_word("讚")

[('感動', 0.9222820997238159),
 ('帥', 0.9205203056335449),
 ('太棒', 0.9156841039657593),
 ('喲', 0.9055087566375732),
 ('哥們', 0.9053220152854919),
 ('水平', 0.9042093753814697),
 ('糟糕', 0.9039602279663086),
 ('快贊', 0.898364245891571),
 ('最愛', 0.8980629444122314),
 ('低', 0.897642970085144)]

In [24]:
#建立詞庫空列表
worddata=[]
#建立將單字對應到索引的字典資料
word2index={}
#建立將索引對應到單字的字典資料
index2word={}
#用來統計最大長度的語句
maxlen=0
for line in data_x:
  if len(line)>maxlen:
    maxlen=len(line)
  for word in line[1:]:
    #如果這個單字不再單字庫內，就新增
    if not word in worddata:
      word2index[word]=len(worddata)
      index2word[len(worddata)]=word
      worddata.append(word)
word_size = len(worddata)
print('單字庫總字數:',word_size)
print('單字庫所有單字:',worddata)
print('最長句子:',maxlen)

單字庫總字數: 11268
單字庫所有單字: ['速度', '太慢', '了', '真是', '不能', '忍了', '好', '脾氣', '全', '被', '磨', '沒', '都', '餓死', '大', '熱天', '送過來', '飯菜', '涼', '？', '該', '怎麼', '評價', '諾大', '的', '盒子', '…', '只', '裝了', '半盒', '3', '塊', '錢', '一', '份', '鵪鶉蛋', '只有', '2', '顆', '我', '好歹', '你', '給', '放了', '問題', '是', '要', '豆皮', '呢', '苦菊', '裡面', '綠色', '菜葉', '用', '根', '筷子', '就', '能', '挑起來', '寬粉', '中指', '那麼', '長', '太', '垃圾', '誰', '點', '你們', '家', '外賣', '死', '全家', '百度', '優惠', '完', '還', '30', '這麼', '不', '說', '去', '湯', '剩', '什麼', '條', '兩', '黑白', '各', '百葉', '有', '支', '組成', '麻辣', '燙', '差勁', '等', '晚上', '也', '送到', '期間', '打', '四', '個', '電話', '店家', '承諾', '取消', '訂單', '最終', '最後', '居然', '確認', '完成', '"', '挺', '快', '菜量', '極少', '正常', '1', '/', '見', '白菜', '一點', '點肉', '肥肉', '極差', '!', '送餐員', '分', '東南西北', '認識', '路', '想', '問問', '我定', '東西', '火星', '嗎', '現在', '餓', '著', '肚子', '這是', '節奏', '根本', '完全', '徹底', '沒有', '送來', '收了', '銀子', '情況', '實在', '跟', '在', '店面', '吃', '感覺', '一樣', '以後', '不會', '吃到', '爆', '~', '真心', '服了', '很', '足', '味道', '行', '就是', '送', '多', '小時'

In [25]:
def text2vector(text):
  #input用來儲存語句的單字向量
  inputs = []
  #將語句以空白字元切割成單字word
  for word in text[1:]:
    #如果這個單字在單字庫內，就將單字轉one hot encode放到input中
    if word in worddata:
      inputs.append(model_w2v.wv.get_vector(word))
  #句子太短做padding
  for i in range(len(inputs),maxlen):
    inputs.insert(0,np.zeros(128))
  return np.array(inputs)

In [27]:
class MyDataset(Dataset):
  def __init__(self,Train=True):
    if Train==True:
      self.data=data_x[:8000]
      self.label=data_y[:8000]
    else:
      self.data=data_x[8000:]
      self.label=data_y[8000:]
  def __getitem__(self, index):
    return torch.tensor(text2vector(self.data[index])), torch.tensor([int(self.label[index])])
  def __len__(self): 
    return len(self.data)

In [28]:
TrainDS=MyDataset(Train=True)
TestDS=MyDataset(Train=False)
TrainDL=DataLoader(dataset=TrainDS,batch_size=5,shuffle=False)
TestDL=DataLoader(dataset=TrainDS,batch_size=len(TestDS),shuffle=False)

In [29]:
class LSTM(nn.Module):
  def __init__(self, InputNode, HiddenNode, LayerNum, OutputNode):
    super(LSTM, self).__init__()
    self.Cell = nn.LSTM(InputNode, HiddenNode, LayerNum, batch_first=True)
    self.fc = nn.Linear(HiddenNode, OutputNode)
  def forward(self, x):
    Cellout, hx = self.Cell(x, None)
    output = torch.sigmoid(self.fc(Cellout[:,-1,:]))
    return output

In [30]:
InputNode=128
HiddenNode=256
LayerNum=1
OutputNode=1
Net=LSTM(InputNode, HiddenNode, LayerNum, OutputNode).to(device)
LossFun= torch.nn.BCELoss()
optim = torch.optim.SGD(Net.parameters(), lr=0.1)
Time_Step = maxlen
Step_L=[]

In [31]:
Net.train()
for epoch in range(500):
  for data,label in TrainDL:
    data= data.reshape(-1,maxlen,InputNode).to(device)
    optim.zero_grad()
    Yh = Net(data.float())
    loss = LossFun(Yh, label.float().to(device))
    Step_L.append(loss.item())
    loss.backward()
    optim.step()
  display.clear_output(wait=True)
  print("回合：",epoch+1,"損失函數",loss.item())

回合： 500 損失函數 0.0015898523852229118
